In [18]:
import pandas as pd
import recbole
import tqdm
from recbole.evaluator.metrics import *

### Version

In [17]:
recbole.__version__

'0.2.1'

### Data preparation

In [16]:
df_pred = pd.read_csv('./preds_full.csv')
df_test = pd.read_csv('./test.csv')

In [13]:
recommendations = {}
pos_len = []
pos_idx_ = []

for user in tqdm.tqdm(sorted(df_pred.user_id.unique())):
    current = (
        df_pred_new[df_pred_new.user_id == user].sort_values('score', ascending=False).relevance > 0
    ).astype(int).tolist()
    
    pos_idx_.append(current)
    pos_len.append(sum(current))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2733/2733 [01:07<00:00, 40.29it/s]


In [19]:
pos_len = np.array(pos_len)

In [20]:
min_len = min(map(lambda x: len(x), pos_idx_))

In [21]:
pos_idx = np.array([x[:min_len] for x in pos_idx_])

In [22]:
pos_idx = np.array(pos_idx)

### Evaluation metrics

Just averaging all user scores:
https://github.com/RUCAIBox/RecBole/blob/master/recbole/evaluator/evaluators.py#L140

In [24]:
results = pd.DataFrame()

In [25]:
results.at["Precision@20","value"] = np.mean(precision_(pos_idx, pos_len)[:,19])

In [26]:
results.at["Recall@20","value"] = np.mean(recall_(pos_idx, pos_len)[:,19])

In [27]:
results.at["NDCG@20","value"] = np.mean(ndcg_(pos_idx, pos_len)[:,19])

In [28]:
results.at["MAP@20","value"] = np.mean(map_(pos_idx, pos_len)[:,19])

In [29]:
results.at["MRR@20","value"] = np.mean(mrr_(pos_idx, pos_len)[:,19])

In [30]:
results.at["HitRate@20","value"] = np.mean(hit_(pos_idx, pos_len)[:,19])

In [34]:
results.at["AUC","value"] = auc_(
    np.array((df_pred_new['relevance'] > 0).astype(int).tolist()),
    np.array(df_pred_new['score'].tolist())
)

In [49]:
pos_rank_sum = np.array(
    [sum(np.where(x)[0]+1) for x in pos_idx]
)
user_len_list = np.array([len(df_pred_new[df_pred_new.user_id == user]) for user in sorted(df_test.user_id.unique())])

In [51]:
results.at["GAUC","value"] = gauc_(
    user_len_list=user_len_list, 
    pos_len_list=pos_len, 
    pos_rank_sum=pos_rank_sum
)

### Results

In [52]:
results

,value
Precision@20,0.057519
Recall@20,0.096321
NDCG@20,0.093264
MAP@20,0.039272
MRR@20,0.186324
HitRate@20,0.474570
AUC,0.686625
GAUC,0.858072
